### import libraries

In [ ]:
import geopandas as gpd
from shapely.geometry import box, Polygon, MultiPolygon


### Read files
- Read gpkg Lucas survey to get the poligons
- Dataset https://data.jrc.ec.europa.eu/dataset/e3fe3cd0-44db-470e-8769-172a8b9e8874#dataaccess

In [ ]:
gdf = gpd.read_file("dataset/l2022_survey_cop_radpoly_attr.gpkg")

### Filter agriculutral data
- Filter the U111 agricultural data found n dataset documentation

In [ ]:
agri = gdf[gdf["lu1_code"] == "U111"]

In [ ]:
# Override the existing CRS (force setting without transformation)
agri.set_crs(epsg=32633, allow_override=True, inplace=True)

# Convert to a different CRS, e.g., WGS84 (EPSG:4326) - Latitude/Longitude
agri = agri.to_crs(epsg=4326)

### Get la toscana region

In [ ]:
# Define a function to reverse the coordinates of the polygons
def reverse_coordinates(geom):
    if geom.is_empty:
        return geom
    elif geom.geom_type == 'Polygon':
        return Polygon([(y, x) for x, y in geom.exterior.coords])
    elif geom.geom_type == 'MultiPolygon':
        return MultiPolygon([Polygon([(y, x) for x, y in poly.exterior.coords]) for poly in geom.geoms])
    else:
        raise ValueError("Geometry type not supported")

# Apply the coordinate reversal to the GeoDataFrame
agri['geometry'] = agri['geometry'].apply(reverse_coordinates)

In [ ]:
toscana_bounding_box = box(6.627, 36.619, 18.784, 47.092) # Example coordinates, adjust as needed

# Convert the bounding box to a GeoDataFrame
bbox_gdf = gpd.GeoDataFrame({'geometry': [toscana_bounding_box]}, crs="EPSG:4326")


# Filter the polygons that intersect with the bounding box
toscana_agri = agri[agri.intersects(bbox_gdf.unary_union)]

### Export poligons

In [ ]:
agri.to_file("dataset/l2022_survey_agri.gpkg", driver="GPKG")
toscana_agri.to_file("dataset/l2022_survey_agri_toscana.gpkg", driver="GPKG")